In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [ ]:
!pip install tflearn

In [ ]:
import numpy
import tflearn
import tensorflow as tf
import random


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving intents.json to intents.json


In [ ]:
import json
with open('intents.json') as file:
    data = json.load(file)

In [ ]:
print(data["intents"])

[{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whatsup bro', 'Hey'], 'responses': ['Hello, Thanks for visiting', 'Good to see you again', 'Hi there, how can I help?', "Cool I'm here for help", 'Hey, Good to see you!'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Got to go'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.', 'See you soon']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'hours', 'patterns': ['What hours are you open?', 'What are your hours?', 'When are you open?'], 'responses': ["We're open every day 9am-9pm", 'Our hours are 9am-9pm every day']}, {'tag': 'payments', 'patterns': ['Do you take credit cards?', 'Do you accept Mastercard?', 'Are you cash only?'], 'responses': ['We accept VISA, Mastercard and AMEX', 'We accept most major

In [ ]:
import pickle

In [ ]:
nltk.download('punkt')

In [ ]:
try:   #saving preprocessed data in pickle file
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []   #tokenized words from pattern
    labels = []  #tags
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)   #tokenizing each pattern
            words.extend(wrds)   #appending all words into lists
            docs_x.append(wrds)
            docs_y.append(intent["tag"])   #to know which tag the word belongs to

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

#One hot encoded list of words into training and output
    words = [stemmer.stem(w.lower()) for w in words if w != "?"]   #removing words like 'fine?', no question marks needed
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)


# building Model

In [ ]:
tf.compat.v1.reset_default_graph()#reseting

net = tflearn.input_data(shape=[None, len(training[0])])   #same length
net = tflearn.fully_connected(net, 8)  #8 neurons for hidden layer
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

In [ ]:
try:
    model.load("model.tflearn")   #loading saved model
except:
  model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
  model.save("model.tflearn")

Training Step: 5999  | total loss: 0.01923 | time: 0.039s
| Adam | epoch: 1000 | loss: 0.01923 - acc: 1.0000 -- iter: 40/45
Training Step: 6000  | total loss: 0.01950 | time: 0.046s
| Adam | epoch: 1000 | loss: 0.01950 - acc: 1.0000 -- iter: 45/45
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


## Forming bag of words for the user typed sentence

In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


In [ ]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
        responses=[]
        results = model.predict([bag_of_words(inp, words)])[0]
        results_index = numpy.argmax(results)   #index with highest probability
        tag = labels[results_index]  #gets tag for that
        if(results[results_index]> 0.7):   #setting up a threshold to produce accurate responses
          for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']
          print(random.choice(responses))
        else:
          print("I didnt get that, try again another question")
       

In [ ]:
chat()

Start talking with the bot (type quit to stop)!
You: hey
Hello, Thanks for visiting
You: what is your name
My Birth day is on 5th October, 2020
You: when is your birthday
My Birth day is on 5th October, 2020
You: name?
Cool I'm here for help
You: india's capital
Narendra Modi
You: whi is chachuuu
I didnt get that, try again another question
You: what is corona
Coronavirus disease (COVID-19) is an infectious disease caused by a newly discovered coronavirus.
You: quarantine period needed?
I didnt get that, try again another question
You: qaurantine period needed for corona patients
Headache, Body pain, Tastelessness, Persisting high body temperature
You: symptoms of corona
Coronavirus disease (COVID-19) is an infectious disease caused by a newly discovered coronavirus.
You: What are symptoms of corona
Coronavirus disease (COVID-19) is an infectious disease caused by a newly discovered coronavirus.
You: is corona fatal
Headache, Body pain, Tastelessness, Persisting high body temperature
Y